# DB 초기화 프로그램 - PowerBi용 데이터 생성

In [1]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)        #dictionary unpacking

In [3]:
import random
random.seed(2021)

- product_table 만들기

In [4]:
# products 테이블 생성 sql
sql ="""
    CREATE TABLE if NOT EXISTS products(
	    pid INT PRIMARY KEY AUTO_INCREMENT,
	    pname VARCHAR(40) NOT NULL,
	    pprice INT UNSIGNED NOT NULL,
	    pcategory VARCHAR(20) NOT NULL,
	    pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [5]:
cur = conn.cursor()
cur.execute(sql)

0

In [6]:
category_names = ['뷰티', '의류', '잡화']
beauty = ['립스틱', '향수', '로션', '스킨', '바디', '크림', '아이', '네일', '헤어', '쿠션']
fashion = ['치마', '바지', '셔츠', '재킷', '아우터', '속옷', '기능성', '코트', '점퍼', '양말']
md = ['가방', '에코백', '파우치', '케이스', '지갑', '구두', '운동화', '샌들', '슬리퍼', '로퍼']

In [7]:
for item in beauty:
    price = random.randint(100,999) * 100
    rate = random.randint(85,98)
    cost = int(round(price*rate / 1000) * 10)
    print(item, price, '메이크업', cost)

립스틱 95600 메이크업 87000
향수 97800 메이크업 92910
로션 65700 메이크업 58470
스킨 35300 메이크업 33540
바디 13500 메이크업 12420
크림 58500 메이크업 54990
아이 16500 메이크업 14850
네일 37600 메이크업 33460
헤어 58600 메이크업 50400
쿠션 26500 메이크업 23060


In [8]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([beauty, fashion, md]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85,98)
        cost = int(round(price*rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

- sales_table 만들기

In [12]:
# sales 테이블 생성 sql
sql ="""
    CREATE TABLE if NOT EXISTS sales(
     sid INT PRIMARY KEY AUTO_INCREMENT,
     sdate DATE NOT NULL,
      scompany VARCHAR(20) NOT NULL,
    	spid INT NOT NULL,
    	sunit INT NOT NULL,
     FOREIGN KEY(spid) REFERENCES products(pid)
) AUTO_INCREMENT=101;
"""

In [13]:
cur = conn.cursor()
cur.execute(sql)

0

In [14]:
calendar = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
company_list = ['A상사', 'B회사', 'C워크', 'D센터', 'E무역']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date, company, pid, unit)

2020-1-6 D센터 16 12
2020-1-7 B회사 13 4
2020-1-1 C워크 34 11
2020-1-18 A상사 28 9
2020-1-30 A상사 28 19
2020-1-24 D센터 16 4
2020-1-2 A상사 24 11
2020-1-2 E무역 23 17
2020-1-31 D센터 22 18
2020-1-8 C워크 30 17


In [15]:
sql = "insert into sales values(default, %s,%s,%s,%s);"
for year in range(2015,2021):
    for month, day in calendar.items():
        for i in range(20):
            date = f'{year}-{month}-{random.randint(1, day)}'
            company = company_list[random.randint(0,4)]
            pid = random.randint(11,40)
            unit = random.randint(1,20)
            cur.execute(sql, (date, company, pid, unit))
    conn.commit()

## 두 테이블을 Join한 view 생성

In [16]:
sql = """
    CREATE VIEW sales_book AS
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost, 
        pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [17]:
cur = conn.cursor()
cur.execute(sql)

0

- 데이터를 sales_book.csv로 저장

In [18]:
sql = 'SELECT * FROM sales_book;'
cur.execute(sql)
results = cur.fetchall()

In [19]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,118,2015-01-02,C워크,25,아우터,의류,21500,16,18920,344000,41280
1,104,2015-01-06,B회사,11,립스틱,뷰티,94900,20,80660,1898000,284800
2,109,2015-01-06,C워크,17,아이,뷰티,57800,1,49710,57800,8090
3,103,2015-01-06,E무역,21,치마,의류,38600,3,35900,115800,8100
4,111,2015-01-07,D센터,29,점퍼,의류,65500,3,57640,196500,23580


In [20]:
del df[3]
del df[0]

In [ ]:
df.columns = ['판매일자', '판매처', '상품명', '카테고리', '단가', '수량', '원가', '매출액', '이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-19,C워크,기능성,의류,55300,13,50880,718900,57460
1,2020-01-24,A상사,아우터,의류,21500,5,18920,107500,12900
2,2020-02-06,E무역,치마,의류,38600,3,35900,115800,8100
3,2020-02-09,C워크,스킨,뷰티,66000,14,58740,924000,101640
4,2020-03-06,B회사,립스틱,뷰티,94900,20,80660,1898000,284800


In [21]:
df.to_csv('sales_book(PowerBI).csv', index=False)

## 마무리

In [22]:
cur.close()
conn.close()